In [1]:
import os 
import tensorflow as tf
from tensorflow.keras import models, layers

In [2]:
#  load a dataset of images from a directory structure
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'DL_Brain_tumor',
    shuffle = True,
    image_size = (256,256),
    batch_size = 32,
    
)

Found 7023 files belonging to 2 classes.


In [3]:
# Split the dataset into training, validation, and test sets
def split_dataset(dataset, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    dataset_size = len(dataset)
    train_size = int(train_ratio * dataset_size)
    val_size = int(val_ratio * dataset_size)

    train_ds = dataset.take(train_size)
    remaining = dataset.skip(train_size)
    val_ds = remaining.take(val_size)
    test_ds = remaining.skip(val_size)

    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = split_dataset(dataset)


In [4]:
# Data normalization
normalization_layer = tf.keras.layers.Rescaling(1. / 255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

print("Data preparation completed!")


Data preparation completed!


In [6]:
from tensorflow.keras.models import Sequential
# Data Augmentation (applies only to the training dataset)
data_augmentation = Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

# Add data augmentation to training dataset
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

# Prefetch data for better performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


In [8]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(len(dataset.class_names), activation='softmax')  # Adjust output layer to match the number of classes
])


# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Change to 'categorical_crossentropy' if using one-hot encoding
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)

Epoch 1/20


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


154/154 ━━━━━━━━━━━━━━━━━━━━ 129s 829ms/step - accuracy: 0.7573 - loss: 1.0243 - val_accuracy: 0.8331 - val_loss: 0.4535
Epoch 2/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 141s 916ms/step - accuracy: 0.8008 - loss: 0.5101 - val_accuracy: 0.8310 - val_loss: 0.4634
Epoch 3/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 142s 924ms/step - accuracy: 0.8044 - loss: 0.5045 - val_accuracy: 0.8274 - val_loss: 0.4593
Epoch 4/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 1328s 9s/step - accuracy: 0.8018 - loss: 0.5061 - val_accuracy: 0.8324 - val_loss: 0.4500
Epoch 5/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 131s 851ms/step - accuracy: 0.8014 - loss: 0.5063 - val_accuracy: 0.8253 - val_loss: 0.4611
Epoch 6/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 144s 935ms/step - accuracy: 0.8035 - loss: 0.4998 - val_accuracy: 0.8274 - val_loss: 0.4567
Epoch 7/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 149s 964ms/step - accuracy: 0.8012 - loss: 0.5027 - val_accuracy: 0.8267 - val_loss: 0.4570
Epoch 8/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 149s 964ms/step - accuracy: 0.8038 - loss: 0.4983

In [9]:
model.evaluate(test_ds)

22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 202ms/step - accuracy: 0.8112 - loss: 0.4854


[0.4897303283214569, 0.807860255241394]

In [10]:
from tensorflow.keras.models import load_model

model.save('my_tumor_model.keras')